# Tutorial 3

An introduction to machine learning using MLJ and the Titanic
dataset. Explains how to train a simple decision tree model and
evaluate it's performance on a holdout set.

MLJ is a *multi-paradigm* machine learning toolbox (i.e., not just
deep-learning).

For other MLJ learning resources see the [Learning
MLJ](https://alan-turing-institute.github.io/MLJ.jl/dev/learning_mlj/)
section of the
[manual](https://alan-turing-institute.github.io/MLJ.jl/dev/).

## Activate package environment

In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, "..", ".."))
Pkg.instantiate()

  Activating project at `~/GoogleDrive/Julia/HelloJulia`


## Establishing correct data representation

In [2]:
using MLJ
import DataFrames

A ["scientific
type"](https://juliaai.github.io/ScientificTypes.jl/dev/) or
*scitype* indicates how MLJ will *interpret* data (as opposed to how
it is represented on your machine). For example, while we have

In [3]:
typeof(3.14)

Float64

we have

In [4]:
scitype(3.14)

Continuous

and also

In [5]:
scitype(3.143f0)

Continuous

In MLJ, model data requirements are articulated using scitypes.

Here are common "scalar" scitypes:

![](assets/scitypes.png)

There are also container scitypes. For example, the scitype of any
vector is `AbstractVector{S}`, where `S` is the scitype of its
elements:

In [6]:
scitype(["cat", "mouse", "dog"])

AbstractVector{Textual} (alias for AbstractArray{Textual, 1})

We'll be using [OpenML](https://www.openml.org/home) to grab the
Titanic dataset:

In [7]:
table = OpenML.load(42638)
df0 = DataFrames.DataFrame(table)
DataFrames.describe(df0)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,pclass,,1,,3,0,"CategoricalValue{String, UInt32}"
2,sex,,female,,male,0,"CategoricalValue{String, UInt32}"
3,age,29.7589,0.42,30.0,80.0,0,Float64
4,sibsp,0.523008,0.0,0.0,8.0,0,Float64
5,fare,32.2042,0.0,14.4542,512.329,0,Float64
6,cabin,,E31,,C148,687,"Union{Missing, CategoricalValue{String, UInt32}}"
7,embarked,,C,,S,2,"Union{Missing, CategoricalValue{String, UInt32}}"
8,survived,,0,,1,0,"CategoricalValue{String, UInt32}"


The `schema` operator summarizes the column scitypes of a table:

In [8]:
schema(df0) |> DataFrames.DataFrame  # converted to DataFrame for better display

,names,scitypes,types
,Symbol,Type,Type
1,pclass,Multiclass{3},"CategoricalValue{String, UInt32}"
2,sex,Multiclass{2},"CategoricalValue{String, UInt32}"
3,age,Continuous,Float64
4,sibsp,Continuous,Float64
5,fare,Continuous,Float64
6,cabin,"Union{Missing, Multiclass{186}}","Union{Missing, CategoricalValue{String, UInt32}}"
7,embarked,"Union{Missing, Multiclass{3}}","Union{Missing, CategoricalValue{String, UInt32}}"
8,survived,Multiclass{2},"CategoricalValue{String, UInt32}"


Looks like we need to fix `:sibsp`, the number of siblings/spouses:

In [9]:
coerce!(df0, :sibsp => Count);

And we'll regard `:survived`, our target variable, as ordered (the
first level is then considered the "false" class)

In [10]:
coerce!(df0, :survived => OrderedFactor)
levels(df0.survived)

2-element Vector{String}:
 "0"
 "1"

The `:cabin` feature has a lot of missing values, and low frequency
for other classes:

In [11]:
import StatsBase
StatsBase.countmap(df0.cabin)

Dict{Union{Missing, CategoricalArrays.CategoricalValue{String, UInt32}}, Int64} with 148 entries:
  "C104"    => 1
  "E50"     => 1
  "D20"     => 2
  "E58"     => 1
  "C46"     => 1
  "D37"     => 1
  "B96 B98" => 4
  "C86"     => 1
  "C106"    => 1
  "A5"      => 1
  "C52"     => 2
  "B19"     => 1
  "C65"     => 2
  "C30"     => 1
  "D48"     => 1
  "B42"     => 1
  "C128"    => 1
  "E38"     => 1
  "E10"     => 1
  "C124"    => 2
  "D21"     => 1
  "C95"     => 1
  "F4"      => 2
  "C45"     => 1
  "A7"      => 1
  "D45"     => 1
  "F G73"   => 2
  "D6"      => 1
  "B77"     => 2
  "C103"    => 1
  "C7"      => 1
  "D19"     => 1
  "E40"     => 1
  "C126"    => 2
  "D36"     => 2
  missing   => 687
  "B58 B60" => 2
  "B5"      => 2
  "F E69"   => 1
  "D33"     => 2
  "A19"     => 1
  "E25"     => 2
  "C2"      => 2
  "D9"      => 1
  "D15"     => 1
  "C50"     => 1
  "C101"    => 1
  "C68"     => 2
  "B41"     => 1
  "D26"     => 2
  "D49"     => 1
  "C54"     => 1
  ⋮         => ⋮

We'll make `missing` into a bona fide class and group all the other
classes into one:

In [12]:
function class(c)
    if ismissing(c)
        return "without cabin"
    else
        return "has cabin"
    end
end

class (generic function with 1 method)

Shorthand syntax would be `class(c) = ismissing(c) ? "without cabin" :
"has cabin"`. Now to transform the whole column:

In [13]:
df0.cabin = map(class, df0.cabin)          # now a `Textual` scitype
coerce!(df0, :class => Multiclass)
schema(df0)

┌──────────┬───────────────────────────────┬────────────────────────────────────────────────
│ names    │ scitypes                      │ types                                         ⋯
├──────────┼───────────────────────────────┼────────────────────────────────────────────────
│ pclass   │ Multiclass{3}                 │ CategoricalValue{String, UInt32}              ⋯
│ sex      │ Multiclass{2}                 │ CategoricalValue{String, UInt32}              ⋯
│ age      │ Continuous                    │ Float64                                       ⋯
│ sibsp    │ Count                         │ Int64                                         ⋯
│ fare     │ Continuous                    │ Float64                                       ⋯
│ cabin    │ Textual                       │ String                                        ⋯
│ embarked │ Union{Missing, Multiclass{3}} │ Union{Missing, CategoricalValue{String, UInt3 ⋯
│ survived │ OrderedFactor{2}              │ CategoricalValue{String, 

## Splitting into train and test sets

Here we split off 30% of our observations into a
lock-and-throw-away-the-key holdout set, called `df_test`:

In [14]:
df, df_test = partition(df0, 0.7, rng=123)
DataFrames.nrow(df)

624

In [15]:
DataFrames.nrow(df_test)

267

## Cleaning the data

Let's constructor an MLJ model to impute missing data using default hyper-parameters:

In [16]:
cleaner = FillImputer()

FillImputer(
  features = Symbol[], 
  continuous_fill = MLJModels._median, 
  count_fill = MLJModels._round_median, 
  finite_fill = MLJModels._mode)

In MLJ a *model* is just a container for hyper-parameters associated
with some ML algorithm. It does not store learned parameters (unlike
scikit-learn "estimators").

We now bind the model with training data in a *machine*:

In [17]:
machc = machine(cleaner, df)

Machine trained 0 times; caches data
  model: FillImputer(features = Symbol[], …)
  args: 
    1:	Source @607 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}, AbstractVector{Union{Missing, Multiclass{3}}}, AbstractVector{OrderedFactor{2}}, AbstractVector{Textual}}}`


And train the machine to store learned parameters there (the column
modes and medians to be used to impute missings):

In [18]:
fit!(machc);

[ Info: Training machine(FillImputer(features = Symbol[], …), …).


We can inspect the learned parameters if we want:

In [19]:
fitted_params(machc).filler_given_feature

Dict{Symbol, Any} with 7 entries:
  :sibsp    => 0
  :pclass   => "3"
  :survived => "0"
  :sex      => "male"
  :age      => 30.0
  :fare     => 14.5
  :embarked => "S"

Next, we apply the learned transformation on our data:

In [20]:
dfc     =  transform(machc, df)
dfc_test = transform(machc, df_test)
schema(dfc)

┌──────────┬──────────────────┬──────────────────────────────────┐
│ names    │ scitypes         │ types                            │
├──────────┼──────────────────┼──────────────────────────────────┤
│ pclass   │ Multiclass{3}    │ CategoricalValue{String, UInt32} │
│ sex      │ Multiclass{2}    │ CategoricalValue{String, UInt32} │
│ age      │ Continuous       │ Float64                          │
│ sibsp    │ Count            │ Int64                            │
│ fare     │ Continuous       │ Float64                          │
│ cabin    │ Textual          │ String                           │
│ embarked │ Multiclass{3}    │ CategoricalValue{String, UInt32} │
│ survived │ OrderedFactor{2} │ CategoricalValue{String, UInt32} │
└──────────┴──────────────────┴──────────────────────────────────┘


## Split the data into input features and target

The following method puts the column with name equal to `:survived`
into the vector `y`, and everything else into a table (`DataFrame`)
called `X`.

In [21]:
y, X = unpack(dfc, ==(:survived));
scitype(y)

AbstractVector{OrderedFactor{2}} (alias for AbstractArray{OrderedFactor{2}, 1})

While we're here, we'll do the same for the holdout test set:

In [22]:
y_test, X_test = unpack(dfc_test, ==(:survived));

## Choosing an supervised model:

There are not many models that can directly handle a mixture of
scitypes, as we have here:

In [23]:
models(matching(X, y))

4-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name = BetaML, ... )
 (name = DeterministicConstantClassifier, package_name = MLJModels, ... )
 (name = RandomForestClassifier, package_name = BetaML, ... )

This can be mitigated with further pre-processing (such as one-hot
encoding) but we'll settle for one the above models here:

In [24]:
doc("DecisionTreeClassifier", pkg="BetaML")

A simple Decision Tree for classification with support for Missing data, from the Beta Machine Learning Toolkit (BetaML).


In [25]:
Tree = @load DecisionTreeClassifier pkg=BetaML  # model type
tree = Tree()                                   # default instance

[ Info: For silent loading, specify `verbosity=0`. 
import BetaML ✔


DecisionTreeClassifier(
  maxDepth = 0, 
  minGain = 0.0, 
  minRecords = 2, 
  maxFeatures = 0, 
  splittingCriterion = BetaML.Utils.gini, 
  rng = Random._GLOBAL_RNG())

Notice that by calling `Tree` with no arguments we get default
values for the various hyperparameters that control how the tree is
trained. We specify keyword arguments to overide these defaults. For example:

In [26]:
small_tree = Tree(maxDepth=3)

DecisionTreeClassifier(
  maxDepth = 3, 
  minGain = 0.0, 
  minRecords = 2, 
  maxFeatures = 0, 
  splittingCriterion = BetaML.Utils.gini, 
  rng = Random._GLOBAL_RNG())

A decision tree is frequently not the best performing model, but it
is easy to interpret (and the algorithm is relatively easy to
explain). For example, here's an diagramatic representation of a
tree trained on (some part of) the Titanic data set, which suggests
how prediction works:

![](assets/tree.png)

## The fit/predict worflow

We now the bind data to used for training and evaluation to the model
in a machine, just like we did for missing value imputation. In this
case, however, we also need to specify the training target `y`:

In [27]:
macht = machine(tree, X, y)

Machine trained 0 times; caches data
  model: DecisionTreeClassifier(maxDepth = 0, …)
  args: 
    1:	Source @300 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{2}}, AbstractVector{Multiclass{3}}, AbstractVector{Textual}}}`
    2:	Source @648 ⏎ `AbstractVector{OrderedFactor{2}}`


To train using *all* the bound data:

In [28]:
fit!(macht)

[ Info: Training machine(DecisionTreeClassifier(maxDepth = 0, …), …).


Machine trained 1 time; caches data
  model: DecisionTreeClassifier(maxDepth = 0, …)
  args: 
    1:	Source @300 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{2}}, AbstractVector{Multiclass{3}}, AbstractVector{Textual}}}`
    2:	Source @648 ⏎ `AbstractVector{OrderedFactor{2}}`


And get predictions on the holdout set:

In [29]:
p = predict(macht, X_test);

These are *probabilistic* predictions:

In [30]:
p[3]

         UnivariateFinite{OrderedFactor{2}}     
     ┌                                        ┐ 
   0 ┤■■■■■■■■■ 0.2                             
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
     └                                        ┘ 

In [31]:
pdf(p[3], "0")

0.2

We can also get "point" predictions:

In [32]:
yhat = mode.(p)

267-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "1"
 "1"
 "1"
 "0"
 "0"
 "0"
 "1"
 "0"
 "0"
 "0"
 "1"
 "1"
 "1"
 "1"
 "1"
 "0"
 "0"
 "0"
 "1"
 "1"
 "1"
 "0"
 "1"
 "1"
 "0"
 "0"
 ⋮
 "0"
 "1"
 "1"
 "0"
 "1"
 "0"
 "1"
 "1"
 "1"
 "0"
 "0"
 "1"
 "0"
 "1"
 "0"
 "1"
 "0"
 "0"
 "0"
 "0"
 "0"
 "1"
 "0"
 "1"
 "0"
 "0"

We can evaluate performance using a probabilistic measure, as in

In [33]:
log_loss(p, y_test) |> mean

8.307615579043436

Or using a deterministic measure:

In [34]:
accuracy(yhat, y_test)

0.7265917602996255

List all performance measures with `measures()`. Naturally, MLJ
includes functions to automate this kind of performance evaluation,
but this is beyond the scope of this tutorial. See, eg,
[here](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#Getting-Started).

## Learning more

Some suggestions for next steps are
[here](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#Getting-Started).

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*